In [2]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy
import pandas as pd
from surprise.model_selection import cross_validate
import os
from surprise.model_selection import GridSearchCV


In [3]:
def read_ratings(ratings_csv: str, data_dir: str = "/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw") -> pd.DataFrame:
    """
    Lit le fichier CSV contenant les évaluations des films.

    :param ratings_csv: Nom du fichier CSV contenant les évaluations.
    :param data_dir: Répertoire où se trouve le fichier CSV.
    :return: DataFrame contenant les évaluations.
    """
    data = pd.read_csv(os.path.join(data_dir, ratings_csv))
    print("Dataset ratings chargé")
    return data

def read_movies(movies_csv: str, data_dir: str = "/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw") -> pd.DataFrame:
    """
    Lit le fichier CSV contenant les informations sur les films.

    :param movies_csv: Nom du fichier CSV contenant les informations sur les films.
    :param data_dir: Répertoire où se trouve le fichier CSV.
    :return: DataFrame contenant les informations sur les films.
    """
    df = pd.read_csv(os.path.join(data_dir, movies_csv))
    print("Dataset movies chargé")
    return df

def cross_validation_model(df: pd.DataFrame, model):
    reader = Reader(rating_scale = (0.5, 5))
    data = Dataset.load_from_df(df[["userId", "movieId", "rating"]], reader)
    algo = model
    print("Début de la cross_validation")
    print(cross_validate(algo, data, cv=5))
    return algo

In [4]:
ratings = read_ratings('ratings.csv')
movies = read_movies('movies.csv')

df = pd.merge(ratings, movies[['movieId', 'genres', 'title']], on = 'movieId', how = 'left')

df.shape

df.head()

Dataset ratings chargé
Dataset movies chargé


,userId,movieId,rating,timestamp,genres,title
0,1,2,3.5,1112486027,Adventure|Children|Fantasy,Jumanji (1995)
1,1,29,3.5,1112484676,Adventure|Drama|Fantasy|Mystery|Sci-Fi,"City of Lost Children, The (CitÃ© des enfants ..."
2,1,32,3.5,1112484819,Mystery|Sci-Fi|Thriller,Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
3,1,47,3.5,1112484727,Mystery|Thriller,Seven (a.k.a. Se7en) (1995)
4,1,50,3.5,1112484580,Crime|Mystery|Thriller,"Usual Suspects, The (1995)"


In [5]:
# On sélectionne environ 20 % du dataset
new_df = df[df['userId'] <= 600]

new_df.shape

(84762, 6)

In [6]:
from surprise import Reader

from surprise import Dataset

reader = Reader(rating_scale=(0.5, 5))

df_surprise = Dataset.load_from_df(new_df[["userId", "movieId", "rating"]], reader=reader)

In [7]:
# Choix d'un utilisateur pour recommandation
user_id = 540

In [8]:
# Construire le jeu d'entraînement complet à partir du DataFrame df_surprise
train_set = df_surprise.build_full_trainset()

# Initialiser une liste vide pour stocker les paires (utilisateur, film) pour le jeu "anti-testset"
anti_testset = []

# Convertir l'ID de l'utilisateur externe en l'ID interne utilisé par Surprise
targetUser = train_set.to_inner_uid(user_id)

# Obtenir la valeur de remplissage à utiliser (moyenne globale des notes du jeu d'entraînement)
moyenne = train_set.global_mean

# Obtenir les évaluations de l'utilisateur cible pour les films
user_note = train_set.ur[targetUser]

# Extraire la liste des films notés par l'utilisateur
user_film= [item for (item,_) in (user_note)]

# Obtenir toutes les notations du jeu d'entraînement
ratings = train_set.all_ratings()

# Boucle sur tous les items du jeu d'entraînement
for film in train_set.all_items():

    # Si l'item n'a pas été noté par l'utilisateur
    if film not in user_film:
        # Ajouter la paire (utilisateur, films, valeur de remplissage) à la liste "anti-testset"
        anti_testset.append((user_id, train_set.to_raw_iid(film), moyenne))

In [13]:
svd = SVD()

param_grid = {'n_factors': [100,150],
              'n_epochs': [20,25,30],
              'lr_all':[0.005,0.01,0.1],
              'reg_all':[0.02,0.05,0.1]}

grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)

grid_search.fit(df_surprise)

In [14]:
print(grid_search.best_score['rmse'])

print(grid_search.best_score['mae'])

print(grid_search.best_params['rmse'])

tunedSVD = grid_search.best_estimator['rmse']

cross_validate(tunedSVD, df_surprise, measures=['RMSE', 'MAE'], cv=5, verbose=True)

0.8021428231692677
0.6132315699246139
{'n_factors': 150, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.05}
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7942  0.7931  0.7942  0.7927  0.7936  0.7936  0.0006  
MAE (testset)     0.6075  0.6066  0.6073  0.6058  0.6067  0.6068  0.0006  
Fit time          51.08   53.08   55.89   52.79   54.14   53.40   1.59    
Test time         7.84    6.39    9.14    7.77    7.74    7.77    0.87    


{'test_rmse': array([0.79422785, 0.79311891, 0.79416254, 0.79273502, 0.79357568]),
 'test_mae': array([0.60748158, 0.60659135, 0.60731922, 0.60575252, 0.60671391]),
 'fit_time': (51.075568199157715,
  53.07694411277771,
  55.89328098297119,
  52.79136347770691,
  54.13805341720581),
 'test_time': (7.836791276931763,
  6.387066841125488,
  9.137169599533081,
  7.76590371131897,
  7.741337537765503)}